# Comparing Entity Matching models with SDK-experimental demo

This notebook uses time series and assets from the open industrial data tenant. It uses cognite-sdk-python-experimental to fit entity matcher models using different parameter combinations. The data is split into a training and test set. The models are compared using running time and the F1 score (combination of precision and recall) on the test set. To read the definition of precision and reacll see [here](https://en.wikipedia.org/wiki/Precision_and_recall). 

One model is chosen and the false positives (incorrect predicted matches by the model), and false negatives (matches the model were not able to find) are briefly investigated.

## Get access to CDF
We assume you have some basic knowledge of CDF and the SDK. If not, please follow the 'lab' tutorials first.

For this tutorial you need access to the publicdata project / tenant. If you don't have one, you can get an API-key [here](https://content.cognite.com/open-industrial-data).

## Import modules
We need to import some Python modules in order to interact with CDF. We will use the Python SDK with Experimental Extensions, which we below refer to as a client. 

In [ ]:
import random
import time
import matplotlib.pyplot as plt
import pandas as pd
from cognite.client import CogniteClient
from getpass import getpass

## Create a client

When you create the CogniteClient below, getpass will ask for your API key in an extra password field. Simply paste ypu publicdata API-key and press shift+enter.

In [ ]:
project = "your_project"
api_key = getpass("Please enter Open Industrial Data API-KEY: ")
client = CogniteClient(project=project, api_key=api_key, client_name="dshub")

## Get data

In this tutorial we will use time series and assets data.

In [ ]:
time_series = client.time_series.list(limit=1000)

# Split into train and test data

Shuffle the time-series list randomly and use the first 60% as the training set and the rest as the test set.

In [ ]:
train_size = 0.6
num_train_samples = round(len(time_series) * train_size)

In [ ]:
random.seed(1006)
random.shuffle(time_series)
time_series_train = time_series[0:num_train_samples]
time_series_test = time_series[num_train_samples:]

true_matches_train = [(x.get("id"), x.get("asset_id")) for x in time_series_train.dump()]
true_matches_test = [(x.get("id"), x.get("asset_id")) for x in time_series_test.dump()]

true_matches_train = [x for x in true_matches_train if x[1]!=None]
true_matches_test = [x for x in true_matches_test if x[1]!=None]

In [ ]:
assetsIds=list(set([x[1] for x in true_matches_train]))
assets=client.assets.retrieve_multiple(ids=assetsIds)

## Test different parameter combinations

In this tutorial 10 parameter combinations are tested.

In [ ]:
test_combs = [
    {
        "name": "simple",
        "match_fields": [("name", "name")],
        "feature_type": "simple",
        "classifier": "RandomForest",
    },
    {
        "name": "simple_extra_keys",
        "match_fields": [("name", "name"), ("description", "description")],
        "feature_type": "simple",
        "classifier": "RandomForest",
    },
    {
        "name": "bigram",
        "match_fields": [("name", "name")],
        "feature_type": "bigram",
        "classifier": "RandomForest",
    },
    {
        "name": "bigram_extra_keys",
        "match_fields": [("name", "name"), ("description", "description")],
        "feature_type": "bigram",
        "classifier": "RandomForest",
    },
    {
        "name": "simple_all_keys",
        "match_fields": [
            ("name", "name"),
            ("description", "description"),
            ("name", "description"),
            ("description", "name"),
        ],
        "feature_type": "simple",
        "classifier": "RandomForest",
    },
    {
        "name": "frequency_weighted_bigram",
        "match_fields": [("name", "name")],
        "feature_type": "simple",
        "classifier": "RandomForest",
    },
    {
        "name": "frequency_weighted_bigram",
        "match_fields": [("name", "name")],
        "feature_type": "frequency-weighted-bigram",
        "classifier": "RandomForest",
    },
    {
        "name": "bigram_extra_tokenizers",
        "match_fields": [("name", "name")],
        "feature_type": "bigram-extra-tokenizers",
        "classifier": "RandomForest",
    },
    {
        "name": "bigram_combo",
        "match_fields": [("name", "name")],
        "feature_type": "bigram-combo",
        "classifier": "RandomForest",
    },
    {
        "name": "unsupervised",
        "match_fields": [("name", "name")],
        "feature_type": "bigram",
        "classifier": "Unsupervised",
    },
]

In [ ]:
def evaluate(true_matches, predicted_matches, score_threshold):
    """ 
    Evaluate the performance of a model. 
    Find true-postives, false-negative and false-positives gien a score threshold.
    Calculate the precision, recall and F1 score.
    """
    predicted_positive_match = [
        x
        for x in predicted_matches
        if x.get("matches") and x["matches"][0]["score"] > score_threshold
    ]
    predicted_positive_matches = [
        (x["source"]["id"], x["matches"][0]["target"]["id"])
        for x in predicted_positive_match
    ]

    # Calculate precision, recall and f1_beta
    true_positives = [x for x in predicted_positive_matches if x in true_matches]
    false_postives = [x for x in predicted_positive_matches if x not in true_matches]
    false_negatives = [x for x in true_matches if x not in predicted_positive_matches]

    precision = len(true_positives) / (len(true_positives) + len(false_postives))
    recall = len(true_positives) / (len(true_positives) + len(false_negatives))

    f1 = 2 * precision * recall / (precision + recall)
    return (true_positives, false_postives, false_negatives, precision, recall, f1)

Loop through all test combinations. Fit a model using the training data and predict time-series to asset matches for the test data. Use the true matches for the test data to evaluate the performance of the model.
Store run time as the time from the model starts training until it is done predicting. 

In [ ]:
score_threshold = 0.75
for parameter_comb in test_combs:
    if parameter_comb["classifier"] == "Unsupervised":
        true_matches = None
        classifier = None
    else:
        true_matches = true_matches_train
        classifier = parameter_comb["classifier"]

    start_time = time.time()
    model = client.entity_matching.fit(
        sources=time_series_train,
        targets=assets,
        true_matches=true_matches,
        match_fields=parameter_comb["match_fields"],
        feature_type=parameter_comb["feature_type"],
        classifier=classifier,
        ignore_missing_fields=True,
        name=parameter_comb["name"],
    )
    
    results = model.predict(
        sources=time_series_test, targets=assets).result
    parameter_comb["runtime"] = time.time() - start_time
    (tp, fp, fn, precision, recall, f1) = evaluate(
        true_matches=true_matches_test,
        predicted_matches=results["items"],
        score_threshold=score_threshold,
    )
    parameter_comb["precision"], parameter_comb["recall"], parameter_comb["f1"] = (
        precision,
        recall,
        f1,
    )
    parameter_comb["id"] = model.id


## Plot the results

In [ ]:
x = [test_comb["f1"] for test_comb in test_combs]
y = [test_comb["runtime"] for test_comb in test_combs]
labels = [test_comb["name"] for test_comb in test_combs]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(x, y)

ax.set_ylabel("Run time (s)")
ax.set_xlabel("F1 (performance)")
ax.set_title("Compare parameter combinations")

for i, txt in enumerate(labels):
    ax.annotate(txt, (x[i], y[i]))

## Select a model, retrieve and update it and investigate the results

In [ ]:
# Select model by name
id_selected = [
    x["id"] for x in test_combs if x["name"] == "bigram_extra_tokenizers"
][0]

In [ ]:
# Retrive the model
model = client.entity_matching.retrieve(id=id_selected)
model

### Update the description of the model
Updating the model is not yet available in the SDK. It will soon be, but for now we have to use the API.

In [ ]:
model.description = "My chosen model"
model = client.entity_matching.update(model)

### Investigate the results

In [ ]:
# Predict on the test-data and evaluate ones more.
results = model.predict(sources=time_series_test, targets=assets).result
(tp, fp, fn, precision, recall, f1) = evaluate(
    true_matches=true_matches_test,
    predicted_matches=results["items"],
    score_threshold=score_threshold,
)

#### Predicted matches

In [ ]:
predicted_matches_list = []
for x in results["items"]:
    time_series_id = x["source"]["id"]
    time_series_name = x["source"]["name"]
    matches = x["matches"]
    if matches:
        score = matches[0]["score"]
        asset_name = matches[0]["target"]["name"]
    else:
        score, asset_name = "", ""
    predicted_matches_list.append(
        {
            "time_series_id": time_series_id,
            "time_series_name": time_series_name,
            "predicted_asset_name": asset_name,
            "score": score,
        }
    )
df_predicted_matches = pd.DataFrame(predicted_matches_list)
df_predicted_matches.head()

#### False negatives

In [ ]:
df_time_series = time_series_test.to_pandas()[["id", "name", "assetId"]].add_prefix(
    "time_series_"
)
df_assets = assets.to_pandas()[["id", "name"]]

false_negatives = pd.merge(
    pd.DataFrame(fn), df_time_series, how="left", left_on=0, right_on="time_series_id"
)
false_negatives = pd.merge(
    false_negatives,
    df_assets.add_prefix("asset_"),
    how="left",
    left_on=1,
    right_on="asset_id",
)
false_negatives = pd.merge(false_negatives, df_predicted_matches, how="left")
false_negatives.drop(
    [0, 1, "time_series_id", "time_series_assetId", "asset_id"], axis=1, inplace=True
)
false_negatives.head()

## Delete all created models

In [ ]:
id_list = [x["id"] for x in test_combs]
client.entity_matching.delete(id_list)